<a href="https://colab.research.google.com/github/kutluhanNG/MachineLearning/blob/main/Sparse_Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sparse Autoencoders**

By adding the right term to the cost function, the autoencoder is pushed to reduce the number of active neurons in the coding layer.

This forces the autoencoder to represent each input as a combination of a small number of activations. As a result, each neuron in the coding layer typically ends up representing a useful feature.

1) Use sigmoid activation in the coding layer > to constrain the codings to take value between 0 and 1).

2) Use a large coding layer.

3) *Add l1 regularization to the coding layer's activations.*

4) Decoder is just a regular decoder.

In [2]:
import tensorflow as tf

sparse_l1_encoder = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(300, activation="sigmoid"),
    tf.keras.layers.ActivityRegularization(l1=1e-4),

])

sparse_l1_decoder = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(28 * 28),
    tf.keras.layers.Reshape([28, 28]),
])

sparse_l1_ae = tf.keras.Sequential([sparse_l1_encoder, sparse_l1_decoder])

This ActivityRegularization layer just returns its inputs, but as a side effect it adds a training loss equal to the sum of the absolute values of its inputs.This only
affects training. Equivalently, you could remove the ActivityRegularization layer
 and set activity_regularizer=tf.keras.regularizers.l1(1e-4) in the previous
 layer. This penalty will encourage the neural network to produce codings close to
 0, but since it will also be penalized if it does not reconstruct the inputs correctly,
 it will have to output at least a few nonzero values. Using the ℓ1
 norm rather than
 the ℓ2
 norm will push the neural network to preserve the most important codings
 while eliminating the ones that are not needed for the input image (rather than just
 reducing all codings)

**Sparse Autoencoder Based on Kullback-Leibler Divergence**

In [3]:
kl_divergence = tf.keras.losses.kullback_leibler_divergence

class KLDivergenceRegularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, weight, target):
      self.weight = weights
      self.target = target

    def __call__(self, inputs):
      mean_activities = tf.reduce_mean(inputs, axis=0)
      return self.weight * kl_divergence(self.target, mean_activities) + kl_divergence(1. - self.target, 1. - mean_activities)

In [ ]:
kld_reg = KLDivergenceRegularizer(weight=5e-3, target=0.1)
sparse_kl_encoder = tf.keras.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation="relu"),
  tf.keras.layers.Dense(300, activation="sigmoid", activity_regularizer=kld_reg)
])

sparse_kl_decoder = tf.keras.Sequential([
  tf.keras.layers.Dense(100, activation="relu"),
  tf.keras.layers.Dense(28 * 28),
  tf.keras.layers.Reshape([28, 28])
])

sparse_kl_ae = tf.keras.Sequential([sparse_kl_encoder, sparse_kl_decoder])